In [1]:
# azure openai from langchain

In [92]:
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
import os




In [96]:
load_dotenv()

# Variables necesarias para Azure OpenAI
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment = os.getenv("03_MINI_DEPLOYMENT")  # nombre del *deployment*, NO del modelo
api_version = os.getenv("AZURE_OPENAI_API_VERSION")  # Ajusta según la versión de tu Azure OpenAI


# ------------------ 3. Configurar modelo Azure ------------------ #
llm = AzureChatOpenAI(
    openai_api_key=api_key,
    azure_endpoint=api_base,
    deployment_name=deployment,
    api_version=api_version,
    # temperature=0
)

# embeddings
embedding_model = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=api_base,
    api_key=api_key,
    openai_api_version=api_version
)





In [4]:
response = llm.invoke("hola")
print(response.content)

¡Hola! ¿En qué puedo ayudarte hoy?


In [5]:
# test lectura de ficha

from langchain.document_loaders import TextLoader
from langchain_core.runnables import Runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.messages import SystemMessage

In [48]:
# load files
aid_name = "actuación_conjunta_isciii-cdti"
# aid_name = "proyectos_de_i_+_d"
# aid_name = "proyectos_de_i+d_aeroespacial_y_salud"

aid_path = f"data/cdti/cdti-aids/{aid_name}"
description_path = f"{aid_path}/{aid_name}_description.md"
card_path = f"{aid_path}/{aid_name}_card.md"
metadata_path = f"{aid_path}/{aid_name}_metadata.md"

# description
loader = TextLoader(description_path, encoding="utf-8")
docs = loader.load()
description = docs[0].page_content  # asumimos que es un único documento

# card
loader = TextLoader(card_path, encoding="utf-8")
docs = loader.load()
card = docs[0].page_content  # asumimos que es un único documento

# metadata
loader = TextLoader(metadata_path, encoding="utf-8")
docs = loader.load()
metadata = docs[0].page_content  # asumimos que es un único documento

# join
document = description + "\n\n" + card + "\n\n" + metadata


In [49]:
EXTRACTION_SYSTEM_MESSAGE = "Eres un asistente que extrae información de fichas administrativas en español."

EXTRACTION_HUMAN_MESSAGE = """
A continuación tienes el contenido de una ficha técnica de una subvención en formato Markdown.

Devuelve un objeto JSON con las siguientes claves extraídas del texto:
- "organismo": Entidad que da la subvención. Algunos ejemplos son CDTI, SODERCAN, SPRI, etc.
- "nombre": Título de la ayuda o subvención. Suele ser la primera línea del texto
- "linea": Modalidades de la subvención. Solo la ayuda llamada "proyectos de I+D" tiene 5 lineas de ayudas llamadas modalidades. Devolver una lista de python esas 5 modalidades. Esl resto devolver una lista vacía ("[]").
- "fecha_inicio": Fecha de inicio del plazo para presentar la solicitud, en formato dd/mm/yyyy si es posible. Si es todo el año, devolver el string "Todo el año" en fecha inicio y fecha fin.
- "fecha_fin": Fecha de fin del plazo para presentar la solicitud, en formato dd/mm/yyyy si es posible. Si es todo el año, devolver el string "Todo el año" en fecha inicio y fecha fin.
- "objetivo: Objetivo general de la ayuda o actuación. Si existe el parrafo "Objetivo General de la actuación", devolver ese texto. En caso contrario, se construye un breve párrafo de objetivo de la ayuda a partir del texto Usa verbos en formato sustantivo (Creación en vez de crear, apoyo en vez de apoyar, etc).
- "beneficiarios": Beneficiarios de la ayuda o actuación. Si existe el parrafo "Beneficiarios", devolver ese texto. En caso contrario, se construye una frase breve de beneficiarios a partir del texto.
No expliques nada. Solo responde con un JSON válido.
- "anio": Año de la convocatoria. Si no hace referencia al año, devolver el año actual (2025).
- "area": Clasifica la convocatoria en una de las siguientes areas: [I+D, Innovación, Inversión, Internacional]
- "presupuesto_minimo": Extrae el valor del presupuesto mínimo. Si no existe, devolver "".
- "duración_mínima": Extrae la duración mínima de la ayuda en formato string ("<duracion> meses"). Si no existe, devolver "".
- "duración_máxima": Extrae la duración máxima de la ayuda en formato string ("<duracion> meses"). Si aparece una fecha, calcula la duración máxima en meses desde la fecha de inicio de la convocatoria. Si no existe, devolver "".
- "intensidad_del_prestamo": Extrae la información sobre la intensidad del préstamo. Suele ser un porcentaje, y se suele encontrar en el apartado "características de la ayuda". Devuelve el texto asociado, no solo el valor.
- "tipo_financiacion": Extrae el tipo de la ayuda. Suele ser un apartado propio. Si no lo encuentras, devuelve un string vacío.
- "region_de_aplicacion": Busca si la convocatoria tiene restricciones por zona geográfica. si no las tiene, devuelve "No".
- "tipo_de_consorcio": Extrae el tipo de consorcio de la ayuda a partir de la información proporcionada. Si no se encuentra información relativa al tipo de consorcio, devolver un string vacío.
- "link_ficha_tecnica": Extrae la url del apartado "aid url". Si no existe, devolver un string vacío.
- "link_convocatoria": Extrae la url del apartado "doc url". Si no existe, devolver un string vacío.
Contenido:
--------------------
{document}
--------------------
"""

In [50]:
# 4. Prompt que pide varios campos
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=EXTRACTION_SYSTEM_MESSAGE),
    ("human", EXTRACTION_HUMAN_MESSAGE)
])

# 5. Parser JSON para extraer la salida limpia
json_parser = JsonOutputParser()

# 6. Encadenar todo
chain = prompt | llm | json_parser



In [51]:
# 7. Ejecutar cadena con el documento
resultado = chain.invoke({"document": document})

In [52]:
resultado


{'organismo': 'CDTI',
 'nombre': 'Actuación conjunta ISCIII-CDTI',
 'linea': [],
 'fecha_inicio': '10/07/2023',
 'fecha_fin': '28/07/2023',
 'objetivo': 'Impulso del empleo de calidad, fortalecimiento de la salud pública y coordinación de capacidades científicas y empresariales para el desarrollo de productos de diagnóstico clínico y medicamentos estratégicos emergentes.',
 'beneficiarios': 'Empresas.',
 'anio': '2023',
 'area': 'I+D',
 'presupuesto_minimo': '175.000 euros',
 'duración_mínima': '12 meses',
 'duración_máxima': '29 meses',
 'intensidad_del_prestamo': 'Ayuda de hasta el 90% del presupuesto aprobado, siempre que no se superen los límites de intensidad máxima permitidos por la normativa comunitaria de ayudas de estado.',
 'tipo_financiacion': 'Ayuda parcialmente reembolsable.',
 'region_de_aplicacion': 'No',
 'tipo_de_consorcio': '',
 'link_ficha_tecnica': 'https://www.cdti.es/node/180',
 'link_convocatoria': 'https://www.cdti.es/sites/default/files/2023-09/ficha_apr_mision

In [ ]:
# campos para pdf:
# intensidad_de_subvención
# forma_y_plazo_de_cobro
# minimis
# region_de_aplicacion
# tipo_de_consorcio
# costes_elegibles

In [ ]:
# prueba lectura pdf

In [84]:
import os

def obtener_nombre_pdf(directorio):
    for archivo in os.listdir(directorio):
        if archivo.endswith(".pdf"):
            return archivo  # Devuelve el nombre del primer PDF encontrado
    return None  # Si no se encuentra ningún PDF


aid_name = "actuación_conjunta_isciii-cdti"
path_aid = f"data/cdti/cdti-aids/{aid_name}"


pdf_file = obtener_nombre_pdf(path_aid)
pdf_path = path_aid + "/" + pdf_file
print(pdf_path)

data/cdti/cdti-aids/actuación_conjunta_isciii-cdti/ficha_apr_mision_salud_2023.pdf


In [ ]:
# test 1: rag dinamico leyendo pdf con pypdf2

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

loader = PyPDFLoader(pdf_path)
documents = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)

#chroma
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model
)

#qdrant
# client = QdrantClient(":memory:")
# vector_store = QdrantVectorStore(
#     client=client,
#     collection_name="test",
#     embedding=embedding_model,
# )

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)


prompt = ChatPromptTemplate.from_template("""
Use the following pieces of context to answer the question at the end.
If you don't know the answer, say that you don't know.
Always answer in spanish.
Context: {context}
Question: {question}
""")


from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context": retriever,
    "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

ValueError: Collection test not found

In [ ]:
# campos para pdf:
# intensidad_de_subvención
# forma_y_plazo_de_cobro
# minimis
# region_de_aplicacion
# tipo_de_consorcio
# costes_elegibles

aid_intensity_prompt = "Busca información sobre la intensidad de subvención y devuelve el resultado de forma esquemática. Responde directamente, sin añadir texto introductorio o de finalización explicativos."
intensidad_de_subvencion = chain.invoke(aid_intensity_prompt)
print(intensidad_de_subvencion)


• Investigación Industrial  
 – Pequeña Empresa: 70%  
 – Mediana Empresa: 60%  
 – Gran Empresa: 50%  
  • Requisito: Colaboración efectiva (entre empresas, incluyendo al menos una PYME, o entre empresa y organismo de investigación con al menos el 10% de costes subvencionables y derecho a publicar resultados)

• Amplia difusión de los resultados del proyecto  
 – Pequeña Empresa: 80%  
 – Mediana Empresa: 75%  
 – Gran Empresa: 65%

• Desarrollo experimental  
 – Pequeña Empresa: 45%  
 – Mediana Empresa: 35%  
 – Gran Empresa: 25%  
  • Requisito: Colaboración efectiva (entre empresas, incluyendo al menos una PYME, o entre empresa y organismos con condiciones similares a las indicadas)


In [105]:
payment_method_prompt = "Busca información sobre las formas y plazos de cobro de la subvención y devuelve el resultado de forma esquemática. Responde directamente, sin añadir texto introductorio o de finalización explicativos."
forma_y_plazo_de_cobro = chain.invoke(payment_method_prompt)
print(forma_y_plazo_de_cobro)

• Formas de cobro:
  - Se podrá abonar mediante un anticipo, abonado en el momento de la formalización del contrato de préstamo.
  - El pago está condicionado a que el beneficiario esté al corriente en el cumplimiento de las obligaciones tributarias, de la Seguridad Social y de reembolso de ayudas o préstamos previos.

• Plazos:
  - El contrato de préstamo debe formalizarse en un plazo máximo de tres meses a partir de la comunicación de aprobación de la ayuda.
  - Si la formalización no se realiza en dicho plazo por causas imputables a la beneficiaria, la concesión de la ayuda se considerará revocada.
  - Con solicitud debidamente justificada, se podrá autorizar una prórroga adicional máxima de un mes para la formalización del contrato.


In [ ]:
payment_method_prompt = "Busca si la subvención aparece información sobre disponibilidad de minimis. Responde únicamente 'Si' o 'No'."
minimis = chain.invoke(payment_method_prompt)
print(minimis)

No


In [116]:
application_region_prompt = "Busca información sobre la región de aplicación de la subvención, donde tiene que estar localizada geográficamente la empresa candidata, y devuelve el resultado de forma esquemática. Responde directamente, sin añadir texto introductorio o de finalización explicativos."
region_de_aplicacion = chain.invoke(application_region_prompt)
print(region_de_aplicacion)

• No se especifica en los documentos una región geográfica concreta en la que deba estar ubicada la empresa candidata.  
• Únicamente se establece la necesidad de incluir el domicilio fiscal del solicitante y la ubicación del proyecto en la solicitud de ayuda.


In [117]:
consortium_type_prompt = "Busca información sobre el tipo de consorcio de la subvención (numero de empresas aceptables o colaboradoras) y devuelve el resultado de forma concisa. Responde directamente, sin añadir texto introductorio o de finalización explicativos."
tipo_de_consorcio = chain.invoke(consortium_type_prompt)
print(tipo_de_consorcio)

El consorcio se puede conformar con una empresa y hasta un máximo de 6 entidades participantes en total (incluyendo centros y organismos de investigación). Si participan varias empresas, una de ellas actuará como coordinadora.


In [119]:
eligible_costs_prompt = "Busca información sobre los costes elegibles de la subvención y devuelve el resultado de forma esquemática. Responde directamente, sin añadir texto introductorio o de finalización explicativos."
costes_elegibles = chain.invoke(eligible_costs_prompt)
print(costes_elegibles)

• Gastos de personal:  
  - Investigadores, técnicos y personal auxiliar dedicados al proyecto.

• Costes de instrumental y material:  
  - Costes de adquisición y, en su caso, la amortización proporcional al periodo de uso en el proyecto.

• Investigación contractual, conocimientos y patentes:  
  - Adquiridas u obtenidas por licencia de fuentes externas a precios de mercado.

• Consultoría y servicios equivalentes:  
  - Destinados de manera exclusiva al proyecto (excluidas las actividades asociadas a la solicitud).

• Gastos generales suplementarios:  
  - Directamente derivados del proyecto de investigación.

• Otros gastos de explotación:  
  - Incluyen costes de material, suministros y productos directamente derivados del proyecto.

• Gasto derivado del informe del auditor:  
  - Límite máximo de 2.000 euros por beneficiario e hito.

• Leasing:  
  - Elegible si el activo se queda en la empresa, tratándose como amortización;  
  - Se excluye el renting por considerarse un alquile

In [80]:
import pdfplumber

def page_elements_ordered_by_top(page):
    # Extraer bloques de texto (usamos extract_words con joins)
    words = page.extract_words(use_text_flow=True, keep_blank_chars=True)
    text_blocks = []

    if words:
        block = ""
        last_bottom = None
        for w in words:
            if last_bottom and abs(w['top'] - last_bottom) > 10:
                if block.strip():
                    text_blocks.append({"top": last_bottom, "type": "text", "content": block.strip()})
                block = ""
            block += w['text'] + " "
            last_bottom = w['bottom']
        if block.strip():
            text_blocks.append({"top": last_bottom, "type": "text", "content": block.strip()})

    # Extraer tablas
    tables = page.extract_tables()
    table_blocks = []
    for table in tables:
        if table:
            # Estimamos la posición vertical usando el primer valor visible de la tabla
            table_blocks.append({"top": page.bbox[1] + 1, "type": "table", "content": table})

    # Combinar y ordenar por posición
    return sorted(text_blocks + table_blocks, key=lambda x: x["top"])

def pdf_to_markdown_structured(pdf_path):
    markdown_output = ""

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            markdown_output += f"\n\n## Página {i + 1}\n\n"
            elements = page_elements_ordered_by_top(page)

            for element in elements:
                if element["type"] == "text":
                    markdown_output += element["content"] + "\n\n"
                elif element["type"] == "table":
                    table = element["content"]
                    headers = table[0]
                    markdown_output += "| " + " | ".join(headers) + " |\n"
                    markdown_output += "| " + " | ".join(["---"] * len(headers)) + " |\n"
                    for row in table[1:]:
                        markdown_output += "| " + " | ".join(row or [""] * len(headers)) + " |\n"
                    markdown_output += "\n"

    return markdown_output

In [81]:
import os

def obtener_nombre_pdf(directorio):
    for archivo in os.listdir(directorio):
        if archivo.endswith(".pdf"):
            return archivo  # Devuelve el nombre del primer PDF encontrado
    return None  # Si no se encuentra ningún PDF

In [82]:
aid_name = "actuación_conjunta_isciii-cdti"
path_aid = f"data/cdti/cdti-aids/{aid_name}"


pdf_file = obtener_nombre_pdf(path_aid)
pdf_path = path_aid + "/" + pdf_file
print(pdf_path)

data/cdti/cdti-aids/actuación_conjunta_isciii-cdti/ficha_apr_mision_salud_2023.pdf


In [83]:


markdown = pdf_to_markdown_structured(pdf_path)
with open("salida.md", "w", encoding="utf-8") as f:
    f.write(markdown)
print("✅ Markdown generado en 'salida.md'")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

✅ Markdown generado en 'salida.md'
